TODO

- [ ] Organize

In [1]:
from datasets import load_dataset
from typing import Literal

/home/santi/current-projects/chappie/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
output_results_path = "/home/santi/current-projects/chappie/data/agent_experiments/"

In [2]:
gaia_dataset = load_dataset("gaia-benchmark/GAIA", "2023_level1")

Docs suggest to focus on validation set for dev purposes, let's explore it

In [3]:
dev_set = gaia_dataset["validation"]

In [4]:
dev_set

Dataset({
    features: ['task_id', 'Question', 'Level', 'Final answer', 'file_name', 'file_path', 'Annotator Metadata'],
    num_rows: 53
})

### Gather Level 1 Dev Questions

In [5]:
import pandas as pd

In [6]:
df_dev = pd.DataFrame(dev_set)

In [7]:
df_dev.head()

,task_id,Question,Level,Final answer,file_name,file_path,Annotator Metadata
0,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,,{'Steps': '1. Googled Eliud Kipchoge marathon ...
1,8e867cd7-cff9-4e6c-867a-ff5ddc2550be,How many studio albums were published by Merce...,1,3,,,{'Steps': '1. I did a search for Mercedes Sosa...
2,ec09fa32-d03f-4bf8-84b0-1f16922c3ae4,Here's a fun riddle that I think you'll enjoy....,1,3,,,{'Steps': 'Step 1: Evaluate the problem statem...
3,5d0080cb-90d7-4712-bc33-848150e917d3,What was the volume in m^3 of the fish bag tha...,1,0.1777,,,"{'Steps': '1. Searched '""Can Hiccup Supply Eno..."
4,a1e91b78-d3d8-4675-bb8d-62741b4b68a6,In the video https://www.youtube.com/watch?v=L...,1,3,,,{'Steps': '1. Navigate to the YouTube link. 2....


In order to start our dev phase, let's observe how does our React Agent perform of a single question

In [8]:
sample_questions = df_dev.sample(12)

In [9]:
# Dataset copy just for eval

results_df = sample_questions.copy()[["Question", "Final answer"]]
results_df["Agent response"] = None
results_df["is_correct"] = None  # 1 if it is correct, 0 otherwise

results_df = results_df[["Question", "Agent response", "Final answer", "is_correct"]]

In [10]:
results_df

,Question,Agent response,Final answer,is_correct
24,How many slides in this PowerPoint presentatio...,None,4,None
0,If Eliud Kipchoge could maintain his record-ma...,None,17,None
40,"According to Girls Who Code, how long did it t...",None,22,None
31,"In the Scikit-Learn July 2017 changelog, what ...",None,BaseLabelPropagation,None
45,"On June 6, 2023, an article by Carolyn Collins...",None,80GSFC21M0002,None
52,What is the first name of the only Malko Compe...,None,Claus,None
13,Under DDC 633 on Bielefeld University Library'...,None,Guatemala,None
25,"You are Van Helsing, a renowned vampire hunter...",None,100,None
22,You are a telecommunications engineer who want...,None,3,None
3,What was the volume in m^3 of the fish bag tha...,None,0.1777,None


### React Agent Developing

In [11]:
import os
from random import sample

os.sys.path.append("../src")
os.sys.path.append("../src/agents")

import react  # My AI assistant

In [12]:
#react.run_app(user_query="Calculate the result of: (12 multiplied by 3) minus (15 divided by 5) plus (8 added to 2).")

Can I create a column agent_answer in order to evaluate? (i.e. running multiple questions in parallel)

In [ ]:
def eval_answer(row: pd.Series) -> Literal[0, 1]:   
    """
    Evaluate Agent responses of GAIA-like answers. Exact match is mandatory for good responses

    Parameters
    ----------
    model_response: str
        Model response to the question
    gt_answer: str
        Ground truth answer to the question
    
    Returns:
        Literal[0, 1]: 0 if the answer is not correct, 1 otherwise 
    
    Example:
        >>> eval_answer(32.0, 32.1)
        '0'
    """
    model_response = row["Agent response"]
    gt_answer = row["Final answer"]
    return 1 if (model_response == gt_answer) else 0

In [ ]:
def get_agent_response(row) -> str:
    user_query = row["Question"]
    agent_response = react.run_app(user_query=user_query)
    agent_response = str(agent_response)
    return agent_response

In [15]:
# Save agent responses and their evaluation

results_df["Agent response"] = results_df.apply(func=get_agent_response, axis=1)
results_df["is_correct"] = results_df.apply(func=eval_answer, axis=1)

In [16]:
# issue: Ground truth answers have str type 

In [17]:
results_df

,Question,Agent response,Final answer,is_correct
24,How many slides in this PowerPoint presentatio...,I cannot access or analyze the content of a Po...,4,0
0,If Eliud Kipchoge could maintain his record-ma...,"To solve this problem, we need to follow these...",17,0
40,"According to Girls Who Code, how long did it t...",The percentage of computer scientists that wer...,22,0
31,"In the Scikit-Learn July 2017 changelog, what ...",`BaseForest`,BaseLabelPropagation,0
45,"On June 6, 2023, an article by Carolyn Collins...",The work performed by R. G. Arendt was support...,80GSFC21M0002,0
52,What is the first name of the only Malko Compe...,Gianluigi,Claus,0
13,Under DDC 633 on Bielefeld University Library'...,"""Japan""",Guatemala,0
25,"You are Van Helsing, a renowned vampire hunter...",99,100,0
22,You are a telecommunications engineer who want...,2,3,0
3,What was the volume in m^3 of the fish bag tha...,0.5,0.1777,0


In [21]:
xp_name = "initial_experiment"
results_df.to_csv(output_results_path + xp_name + ".csv", index=False)

In [18]:
accuracy = results_df["is_correct"].mean()

In [19]:
(100 * accuracy).astype("str")[:3] + " %"

'0.0 %'

In [52]:
path = output_results_path + "historical.csv"

performance_df = pd.read_csv(path)

results = {
    "iteration": 1,
    "agent": "React agent",
    "tools": "Aritmetic tool",
    "accuracy": results_df.copy()["is_correct"].mean(),
}

performance_df = pd.concat([performance_df, pd.DataFrame([results])], ignore_index=True)
performance_df.to_csv(path, index=False)

How to access and handle this dataset?

### Main Questions to solve

$\square$ Which are the core tools for each level of questions 
  - Level 1:
  - Level 2:
  - Level 3:

